In [1]:
import sys  
!{sys.executable} -m pip install contractions

     |████████████████████████████████| 317kB 6.0MB/s 
     |████████████████████████████████| 245kB 10.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81699 sha256=4308e1e703bc4d5fbfe39e913601b54b8240cfd97ff388fe310cbd1417c705cf
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [2]:
import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(5000)

1000


In [3]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

True

In [4]:
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.layers import Input
from keras.layers.merge import Concatenate
import pandas as pd
import numpy as np
import re
import pickle

In [5]:
def remove_long(sent):
    sentence = []
    for word in sent.split():
        if len(word)>15:
            continue
        else:
            sentence.append(word)
    return ' '.join(sentence)

In [6]:
toxic_comments = pd.read_csv('preprocessed_text.csv')
toxic_comments.dropna(inplace=True)
toxic_comments['comment_text'] = toxic_comments['comment_text'].map(lambda com : remove_long(com))

In [7]:
labels = toxic_comments[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]

In [8]:
X_train , X_test, y_train, y_test= train_test_split(toxic_comments['comment_text'], labels, test_size= 0.1)

In [9]:
vect = Tokenizer()
vect.fit_on_texts(X_train)
vocab_size = len(vect.word_index) + 1
print(vocab_size)

121424


In [10]:
encoded_docs_train = vect.texts_to_sequences(X_train)
max_length = vocab_size
padded_docs_train = pad_sequences(encoded_docs_train, maxlen=1000, padding='post')
print(padded_docs_train)

[[ 5253    59    25 ...     0     0     0]
 [  554    25    57 ...     0     0     0]
 [    6     7   134 ...     0     0     0]
 ...
 [   97 33069     2 ...     0     0     0]
 [  144   114   231 ...     0     0     0]
 [ 1715    70    16 ...     0     0     0]]


In [11]:
encoded_docs_test =  vect.texts_to_sequences(X_test)
padded_docs_test = pad_sequences(encoded_docs_test, maxlen=1000, padding='post')

In [12]:
model = Sequential()
# Configuring the parameters
model.add(Embedding(vocab_size, output_dim=50, input_length=1000))
model.add(LSTM(128, return_sequences=True))  
# Adding a dropout layer
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(6, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 50)          6071200   
_________________________________________________________________
lstm (LSTM)                  (None, 1000, 128)         91648     
_________________________________________________________________
dropout (Dropout)            (None, 1000, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
Total params: 6,212,646
Trainable params: 6,212,646
Non-trainable params: 0
______________________________________________

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy')
history = model.fit(padded_docs_train, y_train,
                    class_weight=None,
                    epochs=3,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[])

Epoch 1/3
4038/4038 [==============================] - 7112s 2s/step - loss: 0.1478 - val_loss: 0.1388
Epoch 2/3
4038/4038 [==============================] - 7783s 2s/step - loss: 0.1436 - val_loss: 0.1384
Epoch 3/3
 925/4038 [=====>........................] - ETA: 1:37:45 - loss: 0.1430

In [ ]:
score = model.predict(X_test)